In [2]:
from dotenv import load_dotenv
from pydantic import BaseModel, GetJsonSchemaHandler
from pydantic.json_schema import JsonSchemaValue
from typing import Any, Optional
import pandas as pd
import matplotlib.pyplot as plt
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage


load_dotenv()

from fast_flights import FlightData, Passengers, Result, get_flights
from pprint import pprint
import requests
import os 


In [3]:
from fast_flights import FlightData, Passengers, Result, get_flights
from pprint import pprint

result: Result = get_flights(
    flight_data=[
        FlightData(date="2025-05-01", from_airport="PHL", to_airport="PHX")
    ],
    trip="one-way",
    seat="economy",
    passengers=Passengers(adults=2, children=1, infants_in_seat=0, infants_on_lap=0),
    fetch_mode="fallback",
)

def format_flight(flight: FlightData):
    return {
        "name": flight.name,
        "departure": flight.departure,
        "arrival": flight.arrival,
        "duration": flight.duration,
        "stops": flight.stops,
        "price": flight.price
    }
pprint([format_flight(flight) for flight in result.flights if flight.is_best])

# The price is currently... low/typical/high
print("The price is currently", result.current_price)

[{'arrival': '2:06 PM on Thu, May 1',
  'departure': '8:41 AM on Thu, May 1',
  'duration': '8 hr 25 min',
  'name': 'Frontier',
  'price': '$297',
  'stops': 1},
 {'arrival': '1:28 PM on Thu, May 1',
  'departure': '5:15 AM on Thu, May 1',
  'duration': '11 hr 13 min',
  'name': 'Spirit',
  'price': '$407',
  'stops': 1},
 {'arrival': '11:42 AM on Thu, May 1',
  'departure': '9:27 AM on Thu, May 1',
  'duration': '5 hr 15 min',
  'name': 'American',
  'price': '$866',
  'stops': 0}]
The price is currently typical


In [11]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console

In [5]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini-2024-07-18")


In [6]:
async def lookup_hotel(city_code: str) -> str:
    # fetch access token
    token_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    client_id = os.getenv("AMADEUS_API_KEY")
    client_secret = os.getenv("AMADEUS_SECRET")
    print(client_id)
    print(client_secret)

    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

    header = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    # Make the token request
    response = requests.post(token_url, data=data, headers=header)
    if response.status_code != 200:
        print("Failed to obtain access token:", response.status_code)
        print(response.text)
        return
    token_info = response.json()
    access_token = token_info['access_token']
    # print("Access Token:", access_token)

    #make hotel list request (collect all hotels in city)
    api = "https://test.api.amadeus.com"
    hotel_list = "/v1/reference-data/locations/hotels/by-city"
    hotel_offers = "/v3/shopping/hotel-offers"
    auth_header = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": "Bearer " + access_token
    }

    params = {
        "cityCode": city_code
    }

    listings_in_city = requests.get(api+hotel_list, headers=auth_header, params=params).json()
    # print(listings_in_city)
    
    # hotel search for next requests
    
    hotel_search_params = [hotel["hotelId"] for hotel in listings_in_city["data"]]
    # print(hotel_search_params)
    responses = []
    inc = 50
    for i in range(0,len(hotel_search_params), inc):
        bound = min(i+inc, len(hotel_search_params))
        search = requests.get(api + hotel_offers, headers=auth_header, params = {"hotelIds": hotel_search_params[i:bound]}).json()
        # hotel_search_params
        # print(search)    
        if ("data" in search):
            responses += search["data"]

    # search = requests.get(api + hotel_offers, headers=auth_header, params = {"hotelIds": hotel_search_params[0:50]}).json()
    # print(search)
    # print(responses)
    out = [{"prices": [p["price"]["total"] for p in r["offers"]], "name": r["hotel"]["name"]} for r in responses]
    # return out
    return f"Here are some hotels in {city_code}: {[c['name'] for c in out]}."

hotel_agent = AssistantAgent(
        "Hotel_Agent",
        model_client,
        tools=[lookup_hotel],
        description="Helps with hotel booking.",
        system_message="""
        You are a hotel search agent.
        Your only tool is lookup_hotel - this tool only accepts the city code in IATA format (airport codes).
        You only make one search at a time.
        """
    )

In [7]:
from fast_flights import FlightData, Passengers, Result, get_flights

def format_flight(flight: FlightData):
    return {
        "name": flight.name,
        "departure": flight.departure,
        "arrival": flight.arrival,
        "duration": flight.duration,
        "stops": flight.stops,
        "price": flight.price
    }


async def lookup_flight(departure_date: str, return_date: str, source_airport_code: str, destination_airport_code: str) -> str:
    leave_result: Result = get_flights(
    flight_data=[
            FlightData(date=departure_date, from_airport=source_airport_code, to_airport=destination_airport_code),
        ],
        trip="one-way",
        seat="economy",
        passengers=Passengers(adults=1, children=0, infants_in_seat=0, infants_on_lap=0),
        fetch_mode="fallback",
    )

    best_leave = format_flight([flight for flight in leave_result.flights if flight.is_best][0])

    return_result: Result = get_flights(
    flight_data=[
            FlightData(date=return_date, from_airport=destination_airport_code, to_airport=source_airport_code),
        ],
        trip="one-way",
        seat="economy",
        passengers=Passengers(adults=1, children=0, infants_in_seat=0, infants_on_lap=0),
        fetch_mode="fallback",
    )

    best_return = format_flight([flight for flight in return_result.flights if flight.is_best][0])

    res = f"Here is the best flight from {source_airport_code} to {destination_airport_code}:\n"
    for key in best_leave:
        res += f"{key}: {best_leave[key]}\n"
    res += f"Here is the best return flight from {destination_airport_code} to {source_airport_code}:\n"
    for key in best_return:
        res += f"{key}: {best_return[key]}\n"
    return res

print(await lookup_flight("2025-05-01", "2025-05-10", "PHL", "PHX"))

flight_agent = AssistantAgent(
    "Flight_Agent",
    model_client,
    tools=[lookup_flight],
    description="Helps with flight booking.",
    system_message="""
    You are a flight search agent.
    Your only tool is lookup_flight. This tool takes in a departure date in YYYY-MM-DD format, a return date in YYYY-MM-DD format, the source airport code, and the destination code, and returns .
    You may only make one search at a time.
    """
)



Here is the best flight from PHL to PHX:
name: Frontier
departure: 8:41 AM on Thu, May 1
arrival: 2:06 PM on Thu, May 1
duration: 8 hr 25 min
stops: 1
price: $99
Here is the best return flight from PHX to PHL:
name: Frontier
departure: 9:35 AM on Sat, May 10
arrival: 9:15 PM on Sat, May 10
duration: 8 hr 40 min
stops: 1
price: $90



In [8]:
from autogen.agentchat import UserProxyAgent

user = UserProxyAgent(
  name="User_Proxy_Agent",
  human_input_mode="ALWAYS",
  is_termination_msg=lambda x: x.get("content", "").strip().endswith("TERMINATE"),
)

In [9]:
async def add(a: str, b:str) -> float:
        return a+b

budget_agent = AssistantAgent(
        "Budget_Agent",
        model_client,
        tools=[add],
        description="Helps with calculating total cost of the trip.",
        system_message="""
        You are a budget agent.
        You will add up the costs of the departure and return flights and the hotel cost.
        You will also provide an estimate of how much food and activities will cost for the duration of the trip.
        You will return the total projected cost of the trip.
        """
)

In [10]:
google_api_key = os.environ.get("GOOGLE_API_KEY")

headers = {
    "X-Goog-Api-Key": google_api_key,
    "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.googleMapsLinks"
}

url = "https://places.googleapis.com/v1/places:searchText?"


async def lookup_excursion(location: str) -> str:
        data = {"textQuery": f"Excursions and activities near {location}"}
        res = requests.post(url, json=data, headers=headers).json()
        
        res_string = f"Here is a list of activities to do near {location}\n"
        for place in res["places"][:10]:
                res_string += f"Activity Name: {place['displayName']['text']}; Address: {place['formattedAddress']}; Link: {place['googleMapsLinks']['placeUri']}"

        return res_string


async def lookup_food(location: str) -> str:
        data = {"textQuery": f"Food and restaurants near {location}"}
        res = requests.post(url, json=data, headers=headers).json()
        
        res_string = f"Here is a list of restaurants near {location}\n"
        for place in res["places"][:10]:
                res_string += f"Activity Name: {place['displayName']['text']}; Address: {place['formattedAddress']}; Link: {place['googleMapsLinks']['placeUri']}"

        return res_string

excursion_agent = AssistantAgent(
        "Excursion_Agent",
        model_client,
        tools=[lookup_excursion, lookup_food],
        description="Helps with finding excursions and food.",
        system_message="""
        You are a flight search agent.
        Your only two tools are lookup_excursion - use it to find excursions near a location, and lookup_food - use it to find food near a location
        You may only make one search at a time.
        """
)

In [12]:
# async def book_trip() -> str:
#     return "Your trip is booked!"


travel_planning_agent = AssistantAgent(
    "TravelPlanningAgent",
    model_client=model_client,
    description="Helps with travel planning.",
    system_message="""
    You are a travel planning agent,
    Your job is to break down complex tasks into smaller, manageable substasks.
    Your team members are:
    Flight agent: searches for flights
    Hotel agent: searches for hotels
    Excursion agent: searches for activities and food, given a location

    You only plan and delegate tasks. You do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all the tasks are complete, summarize the findings and end with "TERMINATE"
    """
)
    # User Proxy agent: can prompt the user for clarifying questions, if needed



termination = TextMentionTermination("TERMINATE")
team = SelectorGroupChat(
    [travel_planning_agent, hotel_agent, flight_agent, excursion_agent],
    model_client=model_client,
    termination_condition=termination,
)
await Console(team.run_stream(task="I live near New York City. Book me a 3 day trip in Los Angeles. I am leaving on 5/13/2025."))



---------- user ----------
I live near New York City. Book me a 3 day trip in Los Angeles. I am leaving on 5/13/2025.
---------- TravelPlanningAgent ----------
To plan your 3-day trip to Los Angeles from New York City, we will break it down into the following tasks:

1. Flight agent : Search for flights from New York City to Los Angeles for departure on 5/13/2025 and return for 5/16/2025.
2. Hotel agent : Find hotel accommodations in Los Angeles for 3 nights from 5/13/2025 to 5/16/2025.
3. Excursion agent : Research activities and food options in Los Angeles for the duration of the trip (5/13/2025 to 5/16/2025).

Once all the agents complete their tasks, I will summarize the findings.
---------- Flight_Agent ----------
[FunctionCall(id='call_iyxJbLSpyme3TzLShT9ho9Kh', arguments='{"departure_date":"2025-05-13","return_date":"2025-05-16","source_airport_code":"JFK","destination_airport_code":"LAX"}', name='lookup_flight')]
---------- Flight_Agent ----------
[FunctionExecutionResult(conte

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='I live near New York City. Book me a 3 day trip in Los Angeles. I am leaving on 5/13/2025.', type='TextMessage'), TextMessage(source='TravelPlanningAgent', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=144), metadata={}, content='To plan your 3-day trip to Los Angeles from New York City, we will break it down into the following tasks:\n\n1. Flight agent : Search for flights from New York City to Los Angeles for departure on 5/13/2025 and return for 5/16/2025.\n2. Hotel agent : Find hotel accommodations in Los Angeles for 3 nights from 5/13/2025 to 5/16/2025.\n3. Excursion agent : Research activities and food options in Los Angeles for the duration of the trip (5/13/2025 to 5/16/2025).\n\nOnce all the agents complete their tasks, I will summarize the findings.', type='TextMessage'), ToolCallRequestEvent(source='Flight_Agent', models_usage=RequestUsage(prompt_tokens=340, completion_t